# Título: Sección 4 - Búsqueda Semántica con Vectores (El Cerebro de IA)

**Objetivo:** Implementar un motor de búsqueda semántica de extremo a extremo dentro de Databricks. Utilizaremos el SDK de OpenAI para comunicarnos con nuestro endpoint de embeddings (`databricks-gte-large-en`), convertiremos texto en vectores y finalmente usaremos Databricks Vector Search para realizar búsquedas por significado.

**Fases del Taller:**
1.  **Entender la Limitación:** Veremos por qué la búsqueda tradicional con `LIKE` es insuficiente.
2.  **Generar Embeddings:** Usaremos el SDK de OpenAI para convertir texto en vectores de forma profesional.
3.  **Indexar Vectores:** Crearemos un índice de búsqueda vectorial optimizado.
4.  **Realizar Búsquedas Semánticas:** Haremos preguntas en español y obtendremos resultados contextualmente relevantes.

### Parte 1: El Problema con la Búsqueda Tradicional

La búsqueda por palabras clave con el operador `LIKE` es frágil. Si buscas "delincuencia", no encontrará un artículo que hable de "criminalidad" o "inseguridad". Es una búsqueda de coincidencias exactas de texto, no de ideas.

In [0]:
%sql
-- Demostración 1: Búsqueda por la palabra "criminalidad"
-- Esta consulta solo encontrará artículos que contengan exactamente esa cadena de texto.

USE curso_arquitecturas;

SELECT titulo, info FROM noticias_silver
WHERE lower(contenido) LIKE '%criminalidad%'
LIMIT 2;

In [0]:
%sql
-- Demostración 2: Búsqueda por la palabra "delincuencia"
-- Nota: Es probable que devuelva un conjunto de resultados completamente diferente,
-- aunque la intención del usuario sea muy similar a la anterior.

SELECT titulo, info FROM noticias_silver
WHERE lower(contenido) LIKE '%delincuencia%'
LIMIT 2;

### Parte 2: La Solución - Búsqueda por Significado (Semántica)

La búsqueda semántica resuelve este problema. No busca palabras, busca **significado**. Esto se logra mediante una tecnología llamada **Embeddings Vectoriales**.

**Analogía:** Piensa en los colores. Podemos representar el "rojo" con el vector `[255, 0, 0]` y un "rojo anaranjado" con `[255, 69, 0]`. Estos vectores son numéricamente muy cercanos, al igual que los colores. Los embeddings hacen lo mismo pero con el "significado" del texto.

#### ¿Qué es un Embedding Vectorial?

Un embedding es una serie de números (un **vector**) que representa un fragmento de texto en un espacio matemático de alta dimensión. Es generado por un modelo de IA (un Modelo de Lenguaje Grande) que ha sido entrenado para entender las relaciones semánticas entre palabras y frases.

#### Nuestra Plataforma: Databricks Vector Search

Databricks Vector Search es una base de datos vectorial integrada y sin servidor. Se encarga de todo el trabajo pesado:
* Almacena los vectores de manera eficiente.
* Crea un índice optimizado para búsquedas de similitud ultra rápidas.
* Se actualiza automáticamente a medida que los datos en nuestra tabla Delta cambian.

### Paso 1: Preparar el Entorno

Para usar Databricks Vector Search, necesitamos dos prerequisitos:
1.  **Unity Catalog Habilitado:** Para la gobernanza de datos.
2.  **Un Endpoint de Model Serving:** Es el recurso de cómputo que alojará nuestro índice vectorial.

In [0]:
%sql
-- Asumimos que un endpoint llamado 'vector-search-endpoint' ya ha sido creado.
-- También nos aseguramos de estar en la base de datos correcta.
USE curso_arquitecturas;

### Paso 1: Generar Embeddings Directamente con SQL

Gracias a la función `ai_query`, podemos realizar la "vectorización" de nuestro texto en un solo paso de SQL, como parte de la creación de nuestra tabla `noticias_gold`. Este método es altamente optimizado y se ejecuta de forma distribuida en todo el clúster de Spark.

In [0]:
%sql
-- Creación de la Tabla Gold con Embeddings usando ai_query

-- Este comando lee de la tabla silver, aplica el modelo de IA a la columna 'contenido',
-- y guarda el resultado en una nueva tabla 'noticias_gold'.
-- Es un ETL de IA en una sola consulta.

CREATE OR REPLACE TABLE noticias_gold AS
SELECT
  *, -- Mantenemos todas las columnas originales de la tabla silver
  ai_query(
    'databricks-gte-large-en', -- El endpoint de embeddings que está activo en tu workspace
    contenido
  ) AS contenido_vector
FROM noticias_silver
LIMIT 100; -- Limitamos a 100 para que el taller sea rápido

### Paso 2: Verificar la Tabla Enriquecida

Ahora que el trabajo pesado de IA se ha completado, verifiquemos que nuestra nueva tabla `noticias_gold` se haya creado correctamente y contenga la columna de vectores.

In [0]:
%sql
-- Inspeccionar la Tabla Gold
-- La nueva columna 'contenido_vector' contiene el ADN semántico de cada noticia.
DESCRIBE TABLE noticias_gold;

In [0]:
%sql
-- Ver un Vector de Ejemplo
-- Así es como se ve un embedding: un array de números que captura el significado.
SELECT titulo, slice(contenido_vector, 1, 10) as vector_preview FROM noticias_gold LIMIT 1;

### Paso 3: Crear una Función para Calcular la Similitud de Coseno

Como no tenemos un índice, necesitamos una herramienta para comparar dos vectores. Crearemos una **Función Definida por el Usuario (UDF)** en SQL que implemente la fórmula matemática de la similitud de coseno.

In [0]:
%sql
-- Esta función toma dos vectores (arrays de floats) y devuelve un valor entre -1 y 1.
-- Un valor más cercano a 1 significa mayor similitud.
-- ¡ESTA ES LA VERSIÓN CORREGIDA!

CREATE OR REPLACE FUNCTION cosine_similarity(v1 ARRAY<FLOAT>, v2 ARRAY<FLOAT>)
RETURNS FLOAT
RETURN (
  SELECT
    sum(element.v1 * element.v2) / (sqrt(sum(element.v1 * element.v1)) * sqrt(sum(element.v2 * element.v2)))
  FROM
    (SELECT explode(arrays_zip(v1, v2)) AS element)
);

### Paso 4: Realizar la Búsqueda Semántica Manualmente

Ahora, usaremos nuestra nueva función `cosine_similarity` para encontrar las noticias más relevantes.

El proceso tiene dos pasos:
1.  Obtener el vector de nuestra pregunta.
2.  Usar ese vector para compararlo con todos los vectores en la tabla `noticias_gold`.

In [0]:
%sql
-- Paso 4: Búsqueda Semántica Completa en SQL

-- Esta única consulta realiza todo el proceso:
-- 1. Define la pregunta y la convierte en un vector usando una CTE (el bloque 'WITH').
-- 2. Compara ese vector con todos los vectores de las noticias en la tabla 'noticias_gold'.
-- 3. Utiliza nuestra función 'cosine_similarity' para calcular la relevancia.
-- 4. Ordena los resultados y devuelve los 4 más relevantes.

WITH pregunta_vectorizada AS (
  SELECT ai_query(
    'databricks-gte-large-en',
    'esfuerzos de las autoridades contra la delincuencia en Cúcuta'
  ) AS query_vector
)
SELECT
  n.titulo,
  n.info,
  n.enlace,
  cosine_similarity(n.contenido_vector, p.query_vector) AS similitud
FROM
  curso_arquitecturas.noticias_gold AS n,
  pregunta_vectorizada AS p
ORDER BY
  similitud DESC
LIMIT 4;

### Conclusión

¡Felicidades! Has implementado un pipeline de IA completo utilizando las mejores prácticas en Databricks:

1.  **Transformación a Escala:** Usaste **SQL con `ai_query`** para la generación masiva de embeddings.
2.  **Gestión de Infraestructura:** Verificaste y creaste la infraestructura necesaria (el endpoint de Vector Search) mediante código.
3.  **Búsqueda Inteligente:** Construiste y consultaste un índice vectorial para potenciar una aplicación de búsqueda semántica.

Este flujo de trabajo es la base para construir sistemas de recomendación, chatbots (RAG) y otras aplicaciones de IA sobre tus propios datos.